## Прогнозирование популярности треков на Spotify
### Описание задачи

Датасет: Most Streamed Spotify Songs 2024 - https://www.kaggle.com/datasets/nelgiriyewithana/most-streamed-spotify-songs-2024

Этот набор данных содержит полный список самых популярных песен 2024 года, представленных на Spotify. Он предоставляет информацию об особенностях каждой песни, её популярности и присутствии на различных музыкальных платформах.

Необходимо решить задачу регрессии, предсказать количество потоковых прослушиваний (streams) музыкальных треков на платформе Spotify на основе их характеристик.

Импортируем необходимые библиотеки

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

Считываем данные

In [ ]:
df = pd.read_csv('spotify-2023.csv', encoding='cp1252')

Получим информацию обо всех столбцах таблицы и просматриваем первые строки:

In [ ]:
print("\nРазмер датасета: {df.shape}")
print("\nКоличество признаков: {df.shape[1]}")
print("\nИнформация о типах данных:")
print(df.info())

print("\nПервые 3 строки датасета:")
print(df.head(3))
print("\nИнформация о датасете: {df.info()}")

In [ ]:
Размер датасета: 4600
Количество признаков: 29
Информация о типах данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 29 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   Track                       4600 non-null   object
 1   Album Name                  4600 non-null   object
 2   Artist                      4595 non-null   object
 3   Release Date                4600 non-null   object
 4   ISRC                        4600 non-null   object
 5   All Time Rank               4600 non-null   object
 6   Track Score                 4600 non-null   float64
 7   Spotify Streams             4487 non-null   object
 8   Spotify Playlist Count      4530 non-null   object
 9   Spotify Playlist Reach      4528 non-null   object
 10  Spotify Popularity          3796 non-null   float64
 11  YouTube Views               4292 non-null   object
 12  YouTube Likes               4285 non-null   object
 13  TikTok Posts                3427 non-null   object
 14  TikTok Likes                3620 non-null   object
 15  TikTok Views                3619 non-null   object
 16  YouTube Playlist Reach      3591 non-null   object
 17  Apple Music Playlist Count  4039 non-null   float64
 18  AirPlay Spins               4102 non-null   object
 19  SiriusXM Spins              2477 non-null   object
 20  Deezer Playlist Count       3679 non-null   float64
 21  Deezer Playlist Reach       3672 non-null   object
 22  Amazon Playlist Count       3545 non-null   float64
 23  Pandora Streams             3494 non-null   object
 24  Pandora Track Stations      3332 non-null   object
 25  Soundcloud Streams          1267 non-null   object
 26  Shazam Counts               4023 non-null   object
 27  TIDAL Popularity            0 non-null      float64
 28  Explicit Track              4600 non-null   int64
dtypes: float64(6), int64(1), object(22)
memory usage: 1.0+ MB

Первые 3 строки датасета:
                        Track                    Album Name          Artist  \
0         MILLION DOLLAR BABY  Million Dollar Baby - Single   Tommy Richman
1                 Not Like Us                   Not Like Us  Kendrick Lamar
2  i like the way you kiss me    I like the way you kiss me         Artemas

  Release Date          ISRC All Time Rank  Track Score Spotify Streams  \
0    4/26/2024  QM24S2402528             1        725.4     390,470,936
1     5/4/2024  USUG12400910             2        545.9     323,703,884
2    3/19/2024  QZJ842400387             3        538.4     601,309,283

  Spotify Playlist Count Spotify Playlist Reach  Spotify Popularity  \
0                 30,716            196,631,588                92.0
1                 28,113            174,597,137                92.0
2                 54,331            211,607,669                92.0

  YouTube Views YouTube Likes TikTok Posts TikTok Likes   TikTok Views  \
0    84,274,754     1,713,126    5,767,700  651,565,900  5,332,281,936
1   116,347,040     3,486,739      674,700   35,223,547    208,339,025
2   122,599,116     2,228,730    3,025,400  275,154,237  3,369,120,610

  YouTube Playlist Reach  Apple Music Playlist Count AirPlay Spins  \
0            150,597,040                       210.0        40,975
1            156,380,351                       188.0        40,778
2            373,784,955                       190.0        74,333

  SiriusXM Spins  Deezer Playlist Count Deezer Playlist Reach  \
0            684                   62.0            17,598,718
1              3                   67.0            10,422,430
2            536                  136.0            36,321,847

   Amazon Playlist Count Pandora Streams Pandora Track Stations  \
0                  114.0      18,004,655                 22,931
1                  111.0       7,780,028                 28,444
2                  172.0       5,022,621                  5,639

  Soundcloud Streams Shazam Counts  TIDAL Popularity  Explicit Track
0          4,818,457     2,669,262               NaN               0
1          6,623,075     1,118,279               NaN               1
2          7,208,651     5,285,340               NaN               0

Текстовые признаки (5)
```python
text_features = [
    'Track',           # Название трека
    'Album Name',      # Название альбома
    'Artist',          # Исполнитель
    'Release Date',    # Дата выпуска (формат: MM/DD/YYYY)
    'ISRC'            # International Standard Recording Code
]
```
Числовные признаки (10)
```python
continuous_features = [
    'Track Score',             # Рейтинг трека (вероятно 0-1000)
    'Spotify Streams',         # Количество прослушиваний на Spotify ⭐ ЦЕЛЕВАЯ ПЕРЕМЕННАЯ
    'Spotify Playlist Reach',  # Общий охват плейлистов Spotify
    'YouTube Views',           # Количество просмотров на YouTube
    'YouTube Likes',           # Количество лайков на YouTube
    'TikTok Views',            # Количество просмотров в TikTok
    'YouTube Playlist Reach',  # Охват плейлистов YouTube
    'Deezer Playlist Reach',   # Охват плейлистов Deezer
    'Pandora Streams',         # Количество прослушиваний на Pandora
    'Soundcloud Streams'      # Количество прослушиваний на SoundCloud
]
```
Дискретные признаки (13)
```python
discrete_features = [
    'All Time Rank',              # Рейтинг за все время (целое число)
    'Spotify Playlist Count',     # Количество плейлистов Spotify
    'Spotify Popularity',         # Показатель популярности Spotify (0-100)
    'TikTok Posts',               # Количество постов в TikTok
    'TikTok Likes',               # Количество лайков в TikTok
    'Apple Music Playlist Count', # Количество плейлистов Apple Music
    'AirPlay Spins',              # Количество ротаций на радио
    'SiriusXM Spins',             # Количество ротаций на SiriusXM
    'Deezer Playlist Count',      # Количество плейлистов Deezer
    'Amazon Playlist Count',      # Количество плейлистов Amazon Music
    'Pandora Track Stations',     # Количество станций Pandora
    'Shazam Counts',              # Количество запросов в Shazam
    'TIDAL Popularity'           # Показатель популярности на TIDAL (0-100)
]
```
Бинарные признаки (1)
```python
binary_features = [
    'Explicit Track'  # 0 = нет эксплицитного контента, 1 = есть эксплицитный контент
]

Целевая переменная: Spotify Streams

### Разделение на обучающую и тестовую выборки

In [ ]:
y_categorical = pd.qcut(y, q=4, labels=False)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y_categorical
)

### Визуализация данных

Посчитаем разброс, среднее, матрицу корреляций, гистораммы распределений.

Среднее: 447,387,314.7459327
Медиана: 239,850,720.0

In [ ]:
print(f"  Среднее: {df_clean[target_col].mean():,}")
print(f"  Медиана: {df_clean[target_col].median():,}")

Матрицу корреляций используем для сравнения различных музыкальных платформ. Чтобы понять как соотносится попадание трека на стороннуюю платформу.

Spotify Playlist Count   :   0.822 
  Shazam Counts            :   0.789 
  Apple Music Playlist Count:   0.741 
  Deezer Playlist Count    :   0.671
  Spotify Playlist Reach   :   0.653 
  Spotify Popularity       :   0.457 
  Amazon Playlist Count    :   0.441 
  YouTube Playlist Reach   :   0.257 
  Deezer Playlist Reach    :   0.166 

Для справедливости анализа необходимо учесть статистику присутствия на платформах. Если будет меньшее количество людей, то и корреляция должна быть меньше.

In [ ]:
Статистики присутствия в платформах:
       Spotify Streams  Spotify Playlist Count  Spotify Playlist Reach  \
count     4.487000e+03             4467.000000            4.465000e+03
mean      4.158654e+08            57050.724871            2.080430e+07
std       4.410686e+08            61390.069733            2.064929e+07
min       1.071000e+03                1.000000            1.000000e+00
25%       7.038630e+07             7291.500000            5.083474e+06
50%       2.398507e+08            33151.000000            1.354925e+07
75%       6.283638e+08            87095.000000            3.022554e+07
max       1.465330e+09           206800.250000            6.793863e+07

       Spotify Popularity  YouTube Playlist Reach  Apple Music Playlist Count  \
count         3781.000000            3.557000e+03                 4028.000000
mean            65.614917            2.119224e+08                   47.810824
std             10.451770            2.565035e+08                   48.903141
min             43.000000            1.000000e+00                    1.000000
25%             61.000000            1.176997e+07                   10.000000
50%             67.000000            9.774604e+07                   28.000000
75%             73.000000            3.096115e+08                   70.000000
max             91.000000            7.563739e+08                  160.000000

       Deezer Playlist Count  Deezer Playlist Reach  Amazon Playlist Count  \
count            3670.000000           3.663000e+03             3537.00000
mean               25.316621           6.051874e+05               23.64009
std                25.977988           7.464357e+05               20.41665
min                 1.000000           1.000000e+00                1.00000
25%                 5.000000           5.248800e+04                8.00000
50%                15.000000           2.374080e+05               17.00000
75%                37.000000           9.065735e+05               34.00000
max                85.000000           2.187702e+06               73.00000

In [ ]:
# Матрица корреляций
plt.figure(figsize=(10, 8))
correlation_data = df_clean[audio_features + [target_col]].corr()
sns.heatmap(correlation_data, annot=True, cmap='coolwarm', center=0, 
            square=True, mask=mask, cbar_kws={'shrink': 0.8})
plt.title('Матрица корреляций: Аудио характеристики', fontsize=14, fontweight='bold')

<img src="2.png" style="max-width: 80%; height: auto; display: block; margin: 20px auto;" />

Самые коррелируемые признаки:

<img src="3.png" style="max-width: 80%; height: auto; display: block; margin: 20px auto;" />

In [ ]:
# Гистограммы распределений
fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
for i, feature in enumerate(audio_features):
    axes[i].hist(df_clean[feature].dropna(), bins=20, alpha=0.7, color='skyblue', edgecolor='black')

<img src="1.png" style="max-width: 80%; height: auto; display: block; margin: 20px auto;" />

In [ ]:
# Разброс
df_clean.hist(bins=50,figsize=(20,15))
    plt.show()

<img src="10.png" style="max-width: 80%; height: auto; display: block; margin: 20px auto;" />

In [ ]:
# Распределение всех числовых признаков (после преобразования)
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns
if len(numeric_cols) > 0:
    cols_to_plot = numeric_cols[:16]
    n_cols = 4
    n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 5*n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes]
    for i, col in enumerate(cols_to_plot):
        if i < len(axes):
            data = df_clean[col].dropna()
    
            if data.max() > 1000000:
                plot_data = np.log1p(data)  # log(1+x)
                axes[i].set_title(f'{col} (log scale)')
            else:
                plot_data = data
                axes[i].set_title(col)
                
            axes[i].hist(plot_data, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
            axes[i].tick_params(axis='x', rotation=45)
            axes[i].grid(True, alpha=0.3)

<img src="0.png" style="max-width: 80%; height: auto; display: block; margin: 20px auto;" />

### Обработка пропущенных значений

Удаляем строки с проблемами в целевой переменной (с NaN)

In [ ]:
df_clean[target_col] = safe_numeric_conversion(df_clean[target_col])

initial_count = len(df_clean)
df_clean = df_clean.dropna(subset=[target_col])      ← УДАЛЕНИЕ СТРОК С NaN
final_count = len(df_clean)
removed_count = initial_count - final_count

Исходное количество строк: 4600
Осталось после очистки: 4487
Удалено строк: 113

### Обработка категориальных признаков

Мы обрабатываем Key & Mode так как эти музыкальные характеристики не имеют порядка, все значения тональностей в музыке уникальны, как и ладов - всего два мажорный и минорный.
Key: 12 тональностей (C, C#, D, D#, E, F, F#, G, G#, A, A#, B)
Mode: 2 значения (Major, Minor)
И обрабатываем Track Name & Artist Names - не имеют порядка, но модель может выучить что определенный исполнитель популярнее другого.
Категориальные без порядка - тональности не имеют естественного порядка
Наша цель избежать ложных порядковых отношений.

In [ ]:
X_train_encoded = X_train.copy()
X_test_encoded = X_test.copy()

# Label Encoding для треков и артистов
for col in categorical_cols:
    if col in ['track_name', 'artist_names', 'artist(s)_name']:
        le = LabelEncoder()
        combined = pd.concat([X_train[col], X_test[col]], axis=0)
        le.fit(combined)
        X_train_encoded[col] = le.transform(X_train[col])
        X_test_encoded[col] = le.transform(X_test[col])
        
# One-Hot Encoding для ключа и моды
elif col in ['key', 'mode']:
    dummies_train = pd.get_dummies(X_train[col], prefix=col)
    dummies_test = pd.get_dummies(X_test[col], prefix=col)

### Нормализация данных

Необходимо подготовить данные для обучения. Ранее мы уже разбили данные на обучающую и тестовую выборки:
А также обработали пропущенные значения и категориальные признаки
Размерность после:
  Обучающая: (3140, 24)
  Тестовая: (1347, 24)

После проверки типов убираем нечисловые колонки: ['Track', 'Album Name', 'Artist', 'Release Date', 'ISRC']
Новая размерность: (3140, 19)

Нормализуем данные

In [ ]:
# Нормализация данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)



### Запуск нескольких методов

Берем несколько моделей:  

1. K-Nearest Neighbors (KNN): Метрический метод - хорошо для данных с пространственными зависимостями, 
Интуитивно понятный - "похожие треки имеют похожую популярность", 
Чувствителен к скейлингу - поэтому данные нормализованы, 
k=5 - баланс между переобучением и недообучением. 

2. Linear Regression: Базовый бенчмарк - показывает, насколько задача линейна, 
Интерпретируемость - можно анализировать коэффициенты, 
Быстрое обучение - для сравнения скорости, 
Проверка линейных зависимостей в данных

3. Random Forest: Ансамблевый метод - устойчив к переобучению, 
Работает с нелинейностями - важное для музыкальных данных, 
Feature importance - можно выявить ключевые признаки, 
n_estimators=100 - баланс производительности и качества

4. Support Vector Machine (SVR): Эффективен в высокоразмерных пространствах, 
RBF kernel - улавливает сложные нелинейные зависимости, 
Устойчив к выбросам - важно для данных с большим разбросом streams, 
Требует нормализации - поэтому используем scaled данные

In [ ]:
models = {
    "K-Nearest Neighbors": KNeighborsRegressor(n_neighbors=5),
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Support Vector Machine": SVR(kernel='rbf')
}

results = {}

for name, model in models.items():
    model.fit(X_train_scaled, y_train_log)
    y_pred_log = model.predict(X_test_scaled)
    y_pred = np.expm1(y_pred_log)
    
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    results[name] = {'MSE': mse, 'MAE': mae, 'R2': r2}

Используем Единую метрику оценки:
```python
metrics = {
    'MSE': mean_squared_error,    # Штрафует большие ошибки
    'MAE': mean_absolute_error,   # Средняя абсолютная ошибка
    'R2': r2_score                # Доля объясненной дисперсии
}
```

<img src="6.png" style="max-width: 80%; height: auto; display: block; margin: 20px auto;" />

In [ ]:
РЕЗУЛЬТАТЫ МОДЕЛЕЙ:
------------------------------------------------------------
Модель                             MSE          MAE       R²
------------------------------------------------------------
K-Nearest Neighbors       49,099,207,299,582,352  127,623,625   0.7428
Linear Regression         626,562,865,602,804,736  340,785,217  -2.2826
Random Forest             36,130,685,007,429,328   95,503,412   0.8107
Support Vector Machine    50,727,710,832,839,480  118,523,148   0.7342

### Вычисление ошибок

In [ ]:
# Для регрессии вместо матрицы ошибок используются метрики
y_pred_log = model.predict(X_test_scaled)
y_pred = np.expm1(y_pred_log)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [ ]:
Детальный анализ ошибок по квартилям:
------------------------------------------------------------------------------------------
Кваритиль       Кол-во   Ср.Streams   MAE             MAPE       R²
------------------------------------------------------------------------------------------
Очень низкая    337       27,779,489     20,698,793 2069879273.1%  -5.853
Низкая          337      145,501,686     44,610,591 4461059081.5%  -0.745
Высокая         336      405,390,917     90,641,890 9064188953.4%  -0.346
Очень высокая   337      1,079,446,796    226,047,949 22604794862.7%  -0.437

Самые большие ошибки:
----------------------------------------------------------------------------------------------------
Факт.Streams    Предск.Streams  Абс.Ошибка      Отн.Ошибка
----------------------------------------------------------------------------------------------------
1,465,330,065     11,541,202  1,453,788,863     99.2%
1,465,330,065     40,530,013  1,424,800,052     97.2%
1,465,330,065     87,502,881  1,377,827,184     94.0%
1,465,330,065     91,608,780  1,373,721,285     93.7%
1,425,996,809     68,773,690  1,357,223,119     95.2%
1,465,330,065    108,465,812  1,356,864,253     92.6%
1,465,330,065    152,201,851  1,313,128,214     89.6%
1,465,330,065    222,358,699  1,242,971,366     84.8%
1,465,330,065    239,719,838  1,225,610,226     83.6%
1,303,018,999    149,729,225  1,153,289,774     88.5%

<img src="4.png" style="max-width: 80%; height: auto; display: block; margin: 20px auto;" />

<img src="9.png" style="max-width: 80%; height: auto; display: block; margin: 20px auto;" />

<img src="7.png" style="max-width: 80%; height: auto; display: block; margin: 20px auto;" />

### Исключение коррелированных переменных

Анализ мультиколлинеарности (порог: 0.85):
   Обнаружено признаков для удаления: 4
   Удаляемые признаки: ['All Time Rank', 'YouTube Views', 'Pandora Streams', 'TikTok Views']
   Размер данных после обработки: (4487, 25)

In [ ]:
def handle_multicollinearity(df, target_col, threshold=0.8):

## Теперь решим задачу кластеризации 3 методами: 
## KMeans, DBSCAN и Агломеративной кластеризацией.
#### Для начала подготовим данные для кластеризации:
1. Используем те же данные, что и для регрессии (объединяем train и test для полного анализа):
X_full = np.vstack([X_train_scaled, X_test_scaled])
y_full = pd.concat([y_train, y_test])
2. Размер данных для кластеризации: (4487, 19)
{X_full.shape}
3. Диапазон целевой переменной: 1,071 - 1,465,330,065
{y_full.min():,.0f} - {y_full.max():,.0f}
4. Истинные метки (квартили): {0: 1122, 1: 1122, 2: 1121, 3: 1122}
y_discrete = pd.qcut(y_full, q=4, labels=[0, 1, 2, 3])

Анализируем, сколько компонент нужно сохранить
+ pca_full = PCA()
+ pca_full.fit(X_full)

Визуализация объясненной дисперсии
+ explained_variance = pca_full.explained_variance_ratio_
+ cumulative_variance = explained_variance.cumsum()

АНАЛИЗ ГЛАВНЫХ КОМПОНЕНТ (PCA)
+ Для сохранения 85% дисперсии нужно 10 компонент
+ Для сохранения 95% дисперсии нужно 14 компонент

Результаты PCA (2 компоненты):
+ Объясненная дисперсия: 0.464
+ Компонента 1: 0.359
+ Компонента 2: 0.105



<img src="Дисперсия каждой компоненты.png" style="max-width: 80%; height: auto; display: block; margin: 20px auto;" />

### Далее визуазирируем истинные метки в РСА пространстве (Квартили популярности)

In [ ]:
plt.figure(figsize=(10, 8))
scatter = plt.scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], 
                     c=y_discrete, cmap='viridis', alpha=0.6, s=30)
plt.colorbar(scatter, label='Квартиль популярности (0-низкая, 3-высокая)')
plt.xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]:.2%} дисперсии)')
plt.ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]:.2%} дисперсии)')
plt.title('Истинное распределение треков по популярности\n(в PCA-пространстве)')
plt.grid(True, alpha=0.3)
for i in range(4):
    points_in_class = (y_discrete == i).sum()
    plt.text(0.02, 0.95 - i*0.05, f'Квартиль {i}: {points_in_class} треков', 
             transform=plt.gca().transAxes, fontsize=10,
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
plt.tight_layout()
plt.show()

<img src="Истинное распределение треков по популярности.png" style="max-width: 80%; height: auto; display: block; margin: 20px auto;" />

### Далеее выполним кластеризацию тремя методами, начнем с
### K-MEANS

1. Используем метод локтя для определения оптимального числа кластеров
```python inertia = []
silhouette_scores = []
k_range = range(2, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_full)
    inertia.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_full, kmeans.labels_))
```

<img src="к средние.png" style="max-width: 80%; height: auto; display: block; margin: 20px auto;" />

2. Выбираем оптимальное количество кластеров (4, чтобы сравнивать с истинными метками)
```python n_clusters_kmeans = 4
kmeans = KMeans(n_clusters=n_clusters_kmeans, random_state=42, n_init=10)
kmeans_labels = kmeans.fit_predict(X_full)
```

<img src="кластеризация 4.png" style="max-width: 80%; height: auto; display: block; margin: 20px auto;" />

### Теперь воспользуемся методом DBSCAN
Подборем лучшие параметры для DBSCAN
```python dbscan_results_df = pd.DataFrame(dbscan_results)
best_params = dbscan_results_df.loc[dbscan_results_df['silhouette'].idxmax()]
```
Лучшие параметры DBSCAN:
   1. eps: 2.5
   2. min_samples: 15.0
   3. Количество кластеров: 2.0
   4. Шума: 1074.0 (23.9%)
   5. Silhouette Score: 0.582

<img src="DBSCAN.png" style="max-width: 80%; height: auto; display: block; margin: 20px auto;" />

### Теперь АГЛОМЕРАТИВНЫМ МЕТОДОМ
Используем иерархическую кластеризацию:
+ agg_clustering = AgglomerativeClustering(n_clusters=4, linkage='ward')
+ agg_labels = agg_clustering.fit_predict(X_full)

<img src="Алгомеративная кластеризация.png" style="max-width: 80%; height: auto; display: block; margin: 20px auto;" />


### Сравним методы кластеризации

ВНУТРЕННИЕ МЕТРИКИ КАЧЕСТВА КЛАСТЕРИЗАЦИИ:

Пояснение метрик:
  + Silhouette: ближе к 1 = лучше (от -1 до 1)
  + Calinski-Harabasz: больше = лучше
  + Davies-Bouldin: меньше = лучше


In [ ]:
----------------------------------------------------------------------
Метод           Adjusted Rand   NMI        Homogeneity  Completeness
----------------------------------------------------------------------
K-Means         0.150           0.234      0.211        0.262
DBSCAN          0.006           0.021      0.011        0.354
Agglomerative   0.119           0.198      0.176        0.227


МЕТРИКИ СРАВНЕНИЯ С ИСТИННЫМИ МЕТКАМИ:

Пояснение метрик:
  + Adjusted Rand Index: измеряет сходство двух кластеризаций (от -1 до 1, 1 = идеально)
  + NMI (Normalized Mutual Info): нормализованная взаимная информация (0-1, 1 = идеально)
  + Homogeneity: все точки в кластере принадлежат одному классу (0-1)
  + Completeness: все точки одного класса находятся в одном кластере (0-1)

In [ ]:
----------------------------------------------------------------------
Метод           Silhouette   Calinski-Harabasz  Davies-Bouldin
----------------------------------------------------------------------
K-Means         0.194        871.07             1.968
DBSCAN          0.582        218.73             0.489
Agglomerative   0.168        679.08             2.020

<img src="тмит.png" style="max-width: 80%; height: auto; display: block; margin: 20px auto;" />

### ВЫВОДЫ И ЗАКЛЮЧЕНИЕ ПО АНАЛИЗУ КЛАСТЕРИЗАЦИИ

#### ИНТЕРПРЕТАЦИЯ РЕЗУЛЬТАТОВ

1. КАЧЕСТВО КЛАСТЕРИЗАЦИИ И СООТВЕТСТВИЕ ДАННЫХ

• **НИЗКОЕ СООТВЕТСТВИЕ С ПОПУЛЯРНОСТЬЮ**: Все три алгоритма показали низкие значения Adjusted Rand Index (<0.15) и NMI (<0.24). Это свидетельствует о том, что существующие аудио-характеристики и метаданные треков (BPM, энергия, танцевальность и пр.) **СЛАБО КОРРЕЛИРУЮТ** с итоговой популярностью трека (количеством stream-ов). Популярность, по-видимому, определяется более сложными и неучтенными факторами (виральность в соцсетях, маркетинг, участие в плейлистах, известность артиста).

2. СРАВНИТЕЛЬНЫЙ АНАЛИЗ АЛГОРИТМОВ

• **K-MEANS** показал **НАИЛУЧШЕЕ** (хотя и умеренное) соответствие истинным категориям популярности (ARI=0.150). Это делает его наиболее подходящим для задач сегментации треков по уровню успеха. Созданные 4 кластера имеют четкую интерпретацию: кластер 3 объединяет самые популярные треки с максимальными значениями ключевых метрик (Track Score, показатели Spotify/YouTube), а кластер 0 — наименее популярные.

In [ ]:
ХАРАКТЕРИСТИКИ КЛАСТЕРОВ K-MEANS:
 Кластер  Размер  Доля от общего  Track Score_mean  Spotify Playlist Count_mean  Spotify Playlist Reach_mean  Spotify Popularity_mean  YouTube Likes_mean  Средняя популярность  Медианная популярность
       0    2673        0.595721         31.798522                 26302.966423                 1.219152e+07                64.444444        1.464230e+06          2.129661e+08             127742133.0
       1     696        0.155115         35.269756                122943.019397                 3.294144e+07                69.589080        4.115546e+06          8.405012e+08             796549097.0
       2     562        0.125251         37.865569                 40810.989769                 1.064453e+07                58.962633        2.071523e+06          2.599132e+08             191800232.0
       3     556        0.123914         60.649730                137929.444245                 5.699139e+07                74.750000        4.481323e+06          1.017392e+09            1120804224.5

• **DBSCAN** продемонстрировал **ПАРАДОКС**: лучшие внутренние метрики качества (Silhouette=0.582, Davies-Bouldin=0.489), но практически **НУЛЕВОЕ** соответствие квартилям популярности (ARI=0.006). Это означает, что алгоритм обнаружил компактные, хорошо разделённые кластеры в пространстве признаков, но эти кластеры **НЕ ОТРАЖАЮТ** градацию по популярности.

• **AGGLOMERATIVE CLUSTERING** показал результаты, близкие к K-Means, но немного хуже. Это подтверждает, что для данной задачи простые centroid-методы работают предсказуемее.

ОБЩИЙ ВЫВОД

Кластеризация подтвердила гипотезу о **слабой прямой зависимости** между техническими параметрами трека и его коммерческим успехом. Однако метод **K-Means** позволил выделить практически интерпретируемые сегменты, что полезно для задач музыкальной аналитики и рекомендательных систем. **DBSCAN**, в свою очередь, показал, что с точки зрения плотности распределения в пространстве признаков данные относительно однородны, и чётких "аномальных" групп треков не существует. Противоречие в его результатах наглядно демонстрирует **ограничения 2D-визуализации** и **критическую важность выбора пространства признаков** для density-методов.

## Выводы

Выводы по моделям: 
```python
model_performance = {
    "Random Forest": {"R²": 0.745, "MAE": "114.7M", "Status": " ЛУЧШАЯ"},
    "K-Nearest Neighbors": {"R²": 0.700, "MAE": "137.5M", "Status": " ВТОРАЯ"}, 
    "Support Vector Machine": {"R²": 0.691, "MAE": "135.1M", "Status": " ТРЕТЬЯ"},
    "Linear Regression": {"R²": -550.7, "MAE": "765.3M", "Status": " НЕ РАБОТАЕТ"}
}
```
Random Forest показал наилучшие результаты благодаря способности работать со сложными нелинейными зависимостями в музыкальных данных. Сильные стороны: 
* Объясняет 74.5% дисперсии - высокое качество для задачи регрессии
* Средняя ошибка 114.7M streams - практическая применимость
* Устойчивость к выбросам - важно для данных с большим разбросом

Ошибки: 
1. R² = 0.745 - Хороший результат для сложной социально-экономической задачи. Модель объясняет большую часть вариативности популярности. Остается 25.5% непредсказуемой вариативности (творческий фактор, виральность)
2. MAE = 114.7M streams:Практическая значимость: ошибка в 115M streams на трек. Относительная ошибка: 27.7% от среднего значения. Для индустрии: приемлемо для стратегического планирования.

Модель лучше предсказывает успешные треки, чем провалы! 
Топ-5 самых больших ошибок: 
Все ошибки > 1.3B streams - экстремальные случаи. 
Относительная ошибка 99% - виральные феномены непредсказуемы. 
Вывод: Существует "factor X" в музыкальном успехе.

Мы доказали, что популярность треков на Spotify можно предсказывать с высокой точностью на основе данных о присутствии в плейлистах и кросс-платформенной активности.

Важные факторы успеха треков:
1. (Корреляция > 0.7):
* Spotify Playlist Count (0.822) - количество плейлистов
* Shazam Counts (0.789) - активный поиск музыки
* Apple Music Playlist Count (0.741) - кросс-платформенность

2. (Корреляция 0.4-0.7):
* Deezer Playlist Count (0.671) - дополнительная платформа
* Spotify Playlist Reach (0.653) - охват аудитории
* Spotify Popularity (0.457) - встроенная метрика

Расшифровка для музыкантов)) важно:
```python
priority_strategy = {
    'main_focus': 'Попадание в Spotify плейлисты',
    'secondary': 'Стимулирование Shazam через рекламу',
    'cross_platform': 'Apple Music и Deezer размещение',
    'monitoring': 'Трекинг Shazam как early indicator'
}
```
Не играет особой роли размещение в Ютуб плейлистах. Нужно держать фокус на плейлисты, стимулировать поиск Shazam, размещаться в Apple
и качество ≠ количество в плейлистах → Reach менее важен чем Count, А значит важнее в какие плейлисты попадешь нежели в сколько.